In [49]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [50]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W32"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

import random
random.seed(42)

test_q_keys_mapping = {
    "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
    "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
    "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32']
}


# Hyper parameters

In [51]:
survey_name = "American_Trends_Panel_W26"

def skip_exp(filename):
    if all(_ not in filename for _ in ['v11', 'v10']): return True
    if any(_ in filename for _ in ['hcbdeu', 'hcaic']): return True
    if ('v11' in filename) and ('bds' not in filename): return True

    if ('v11' in filename) and ('persona' not in filename): return True
    return False

def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    exp_elements = [_ for _ in filename.split('_') if _ in ['history', 'demo', 'persona', 'v10', 'v11', 'v12', 'hcaic', 'hcbds', 'hcbdeu']]
    final = '_'.join(exp_elements)
    # print(filename, '->', final)
    return final


In [52]:

# survey_name = "American_Trends_Panel_W27"

# def skip_exp(filename):
#     if 'pval' not in filename: return True
#     return False

# def get_exp(filename):
#     if filename[-5:] == '.json': filename = filename[:-5]
#     exp_elements = [_ for _ in filename.split('_') if _ in ['history', 'demo', 'persona', 'pval']]
#     final = '_'.join(exp_elements)
#     # print(filename, '->', final)
#     return final

In [53]:

# survey_name = "American_Trends_Panel_W32"

# def skip_exp(filename):
#     # if 'pval' in filename: return True
#     return False

# def get_exp(filename):
#     if filename[-5:] == '.json': filename = filename[:-5]
#     exp_elements = [_ for _ in filename.split('_') if _ in ['history', 'demo', 'persona', 'v10']]
#     final = '_'.join(exp_elements)
#     # print(filename, '->', final)
#     return final

# Get Accuracy

In [54]:
import json
import os
import pandas as pd
import numpy as np

res = {}
for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
    if skip_exp(filename): continue
    exp = get_exp(filename)
    # exp = filename
    if exp not in res:
        res[exp] = []
        
    with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
        data = json.load(f)
    # res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", f"{sum([_['is_correct'] for _ in data])/len(data):.4f}"]
    res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]


print(res)
df = pd.DataFrame(res)
df = df[sorted(df.columns)]
df = df.transpose()

# df['acc_avg'] = df.apply(lambda x: np.mean(x[[1,3,5]]), axis=1)
# df['acc_var'] = df.apply(lambda x: np.var(x[[1,3,5]]), axis=1)
# df['acc_std'] = df.apply(lambda x: np.std(x[[1,3,5]]), axis=1)
# df['acc_avg%'] = df.apply(lambda x: f"{100*x['acc_avg']:.2f}%", axis=1)
# df['acc_std%'] = df.apply(lambda x: f"{100*np.std(x[[1,3,5]]):.2f}%", axis=1)

acc_df = df
acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
acc_df.sort_values(by='acc', ascending=False)

{'persona': ['=533/1090', 0.4889908256880734], 'history_demo_persona': ['=612/1090', 0.5614678899082569], 'v11_hcbds_demo_persona': ['=563/1090', 0.5165137614678899], 'v11_hcbds_history_persona': ['=610/1090', 0.5596330275229358], 'demo_persona': ['=551/1090', 0.5055045871559632], 'v11_hcbds_persona': ['=528/1090', 0.48440366972477067], 'history_persona': ['=605/1090', 0.555045871559633], 'demo': ['=570/1090', 0.5229357798165137], 'history': ['=594/1090', 0.544954128440367], 'history_demo': ['=617/1090', 0.5660550458715596], 'v11_hcbds_history_demo_persona': ['=617/1090', 0.5660550458715596]}


,acc_cnt,acc
history_demo,=617/1090,0.566055
v11_hcbds_history_demo_persona,=617/1090,0.566055
history_demo_persona,=612/1090,0.561468
v11_hcbds_history_persona,=610/1090,0.559633
history_persona,=605/1090,0.555046
history,=594/1090,0.544954
demo,=570/1090,0.522936
v11_hcbds_demo_persona,=563/1090,0.516514
demo_persona,=551/1090,0.505505
persona,=533/1090,0.488991


# Get 1-WD

In [55]:
import pandas as pd
from src.utils import get_file_from_s3

test_q_keys = test_q_keys_mapping[survey_name]
survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
sub_df = survey_df[survey_df['key'].isin(test_q_keys)]

wd_question_mappings = {row['key']: dict(zip(eval(row['references']), eval(row['option_ordinal']))) for _, row in sub_df.iterrows()}
wd_question_mappings

{'WORLDDANGER_W26': {'We live in a safer world': 1.0,
  'We live in a more dangerous world': 2.0,
  'The world we live in is neither safer nor more dangerous': 1.5},
 'REASONGUNC_W26': {'Major reason': 1.0,
  'Minor reason': 2.0,
  'Not a reason': 3.0},
 'GUNIDENTITY_W26': {'Very important': 1.0,
  'Somewhat important': 2.0,
  'Not too important': 3.0,
  'Not at all important': 4.0},
 'GUNRESPNOKIDSB_W26': {'Essential': 1.0,
  'Important but not essential': 2.0,
  'Not important': 3.0,
  'Should not be done': 4.0},
 'GUNRESPKIDSC_W26': {'Essential': 1.0,
  'Important but not essential': 2.0,
  'Not important': 3.0,
  'Should not be done': 4.0}}

## Question level

In [56]:
import random
from collections import Counter

records = {}
for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
    if skip_exp(filename): continue

    with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
        data = json.load(f)
    
    exp = get_exp(filename)

    res = []
    for entry in data:
        q_idx = entry['q_idx']
        q_key = test_q_keys[q_idx]
        pos_mapping = wd_question_mappings[q_key]
        prediction = entry['prediction']
        gold_answer = entry['gold_answer']
        if gold_answer not in pos_mapping: continue
        if prediction not in pos_mapping:
            prediction = random.choice(list(pos_mapping.keys()))
        pred_pos = pos_mapping[prediction]
        gold_pos = pos_mapping[gold_answer]
        res.append(abs(pred_pos-gold_pos))
    records[exp] = [Counter(res), np.mean(res)]

q_wd_df = pd.DataFrame(records)
q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd"})
q_wd_df

,q_1wd_cnt,q_1wd
demo,"{1.0: 317, 0.0: 570, 0.5: 82, 3.0: 33, 2.0: 83}",0.574194
demo_persona,"{1.0: 331, 0.0: 551, 0.5: 97, 3.0: 25, 2.0: 81}",0.568203
history,"{1.0: 291, 0.0: 592, 0.5: 102, 3.0: 31, 2.0: 69}",0.528111
history_demo,"{1.0: 281, 0.0: 617, 0.5: 96, 3.0: 32, 2.0: 59}",0.500461
history_demo_persona,"{1.0: 281, 0.0: 612, 0.5: 101, 3.0: 30, 2.0: 61}",0.500922
history_persona,"{1.0: 275, 0.0: 605, 0.5: 114, 3.0: 28, 2.0: 63}",0.499539
persona,"{1.0: 329, 0.0: 533, 0.5: 105, 3.0: 29, 2.0: 89}",0.595853
v11_hcbds_demo_persona,"{1.0: 326, 0.0: 563, 0.5: 90, 3.0: 27, 2.0: 79}",0.562212
v11_hcbds_history_demo_persona,"{1.0: 277, 0.0: 617, 0.5: 101, 3.0: 32, 2.0: 58}",0.497235
v11_hcbds_history_persona,"{1.0: 265, 0.0: 610, 0.5: 117, 3.0: 29, 2.0: 64}",0.496313


## Survey level

In [57]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wasserstein_distance


def get_1wd_from_df(df, ref_ord_mapping):
    pred_df = df[df['Type'] == 'Prediction']
    pred_dict = {row['Answer']: row['Count'] for idx, row in pred_df.iterrows() if row['Answer'] != 'Refused'}
    gold_df = df[df['Type'] == 'Gold']
    gold_dict = {row['Answer']: row['Count'] for idx, row in gold_df.iterrows() if row['Answer'] != 'Refused'}

    pred_dict = {k: v for k, v in pred_dict.items() if k in ref_ord_mapping}

    distribution_1 = pred_dict
    distribution_2 = gold_dict

    # Get the union of keys
    all_keys = sorted(list(set(distribution_1.keys()).union(set(distribution_2.keys()))))

    # Create aligned frequency arrays
    freqs_1 = np.array([distribution_1.get(key, 0) for key in all_keys])
    freqs_2 = np.array([distribution_2.get(key, 0) for key in all_keys])

    # Normalize the frequencies to get probabilities
    probs_1 = freqs_1 / freqs_1.sum()
    probs_2 = freqs_2 / freqs_2.sum()

    # # Compute the cumulative distribution functions
    cdf_1 = np.cumsum(probs_1)
    cdf_2 = np.cumsum(probs_2)

    # Compute the 1-Wasserstein distance

    positions = [ref_ord_mapping[k] for k in all_keys]  # Positions in the sorted key space

    wd_distance = wasserstein_distance(positions, positions, u_weights=probs_1, v_weights=probs_2)
    # wd_distance = wasserstein_distance(positions, positions, u_weights=cdf_1, v_weights=cdf_2)
    # print(positions)
    # print(wd_distance)

    return wd_distance


# Load the data
record = {}
for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):

    if skip_exp(filename): continue

    exp = get_exp(filename)
    res = [{"pred": [], "gold": []} for _ in range(5)]
    if exp not in record: record[exp] = []

    with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
        data = json.load(f)

    for entry in data:
        q_idx = entry['q_idx']
        prediction = entry['prediction']
        gold_answer = entry['gold_answer']
        res[q_idx]['pred'].append(prediction)
        res[q_idx]['gold'].append(gold_answer)

    # Create dataframes for visualization
    dataframes = []
    for i in range(5):
        df_pred = pd.Series(res[i]['pred']).value_counts().reset_index()
        df_pred.columns = ['Answer', 'Count']
        df_pred['Type'] = 'Prediction'
        df_pred['Question'] = f'Question {i+1}'
        
        df_gold = pd.Series(res[i]['gold']).value_counts().reset_index()
        df_gold.columns = ['Answer', 'Count']
        df_gold['Type'] = 'Gold'
        df_gold['Question'] = f'Question {i+1}'
        
        dataframes.append(pd.concat([df_pred, df_gold]))

    wd = []
    for q_idx, df in enumerate(dataframes):
        key = test_q_keys[q_idx]
        ref_ord_mapping = wd_question_mappings[key]
        wd.append(get_1wd_from_df(df, ref_ord_mapping))

    record[exp].append(np.mean(wd) * 100)

s_wd_df = pd.DataFrame(record)
s_wd_df = s_wd_df[sorted(s_wd_df.columns)].transpose()

# s_wd_df['wd_avg'] = s_wd_df.apply(lambda x: f"{np.mean(x[[0,1,2]]):.2f}", axis=1)
# s_wd_df['wd_std'] = s_wd_df.apply(lambda x: f"{np.std(x[[0,1,2]]):.2f}", axis=1)
s_wd_df = s_wd_df.rename(columns={0: 's_1wd'})
s_wd_df

,s_1wd
demo,30.927270
demo_persona,40.702975
history,47.505423
history_demo,44.314931
history_demo_persona,37.496797
history_persona,36.211950
persona,46.748929
v11_hcbds_demo_persona,38.765550
v11_hcbds_history_demo_persona,37.209423
v11_hcbds_history_persona,36.865617


# Full Results

In [67]:
merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
# merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
# merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
# merged_df = merged_df.sort_values(by='acc', ascending=False)

# merged_df.sort_values(by='acc_avg', ascending=False)
# merged_df.sort_values(by='wd_avg', ascending=True)
# merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

merged_df

,acc_cnt,acc,q_1wd_cnt,q_1wd
v11_hcbds_history_persona,=610/1090,0.559633,"{1.0: 265, 0.0: 610, 0.5: 117, 3.0: 29, 2.0: 64}",0.496313
v11_hcbds_history_demo_persona,=617/1090,0.566055,"{1.0: 277, 0.0: 617, 0.5: 101, 3.0: 32, 2.0: 58}",0.497235
history_persona,=605/1090,0.555046,"{1.0: 275, 0.0: 605, 0.5: 114, 3.0: 28, 2.0: 63}",0.499539
history_demo,=617/1090,0.566055,"{1.0: 281, 0.0: 617, 0.5: 96, 3.0: 32, 2.0: 59}",0.500461
history_demo_persona,=612/1090,0.561468,"{1.0: 281, 0.0: 612, 0.5: 101, 3.0: 30, 2.0: 61}",0.500922
history,=594/1090,0.544954,"{1.0: 291, 0.0: 592, 0.5: 102, 3.0: 31, 2.0: 69}",0.528111
v11_hcbds_demo_persona,=563/1090,0.516514,"{1.0: 326, 0.0: 563, 0.5: 90, 3.0: 27, 2.0: 79}",0.562212
demo_persona,=551/1090,0.505505,"{1.0: 331, 0.0: 551, 0.5: 97, 3.0: 25, 2.0: 81}",0.568203
demo,=570/1090,0.522936,"{1.0: 317, 0.0: 570, 0.5: 82, 3.0: 33, 2.0: 83}",0.574194
persona,=533/1090,0.488991,"{1.0: 329, 0.0: 533, 0.5: 105, 3.0: 29, 2.0: 89}",0.595853


**For printing:**

In [59]:
merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
merged_df

,acc_cnt,acc,q_1wd_cnt,q_1wd
v11_hcbds_history_persona,'=610/1090,0.559633,"{1.0: 265, 0.0: 610, 0.5: 117, 3.0: 29, 2.0: 64}",0.496313
v11_hcbds_history_demo_persona,'=617/1090,0.566055,"{1.0: 277, 0.0: 617, 0.5: 101, 3.0: 32, 2.0: 58}",0.497235
history_persona,'=605/1090,0.555046,"{1.0: 275, 0.0: 605, 0.5: 114, 3.0: 28, 2.0: 63}",0.499539
history_demo,'=617/1090,0.566055,"{1.0: 281, 0.0: 617, 0.5: 96, 3.0: 32, 2.0: 59}",0.500461
history_demo_persona,'=612/1090,0.561468,"{1.0: 281, 0.0: 612, 0.5: 101, 3.0: 30, 2.0: 61}",0.500922
history,'=594/1090,0.544954,"{1.0: 291, 0.0: 592, 0.5: 102, 3.0: 31, 2.0: 69}",0.528111
v11_hcbds_demo_persona,'=563/1090,0.516514,"{1.0: 326, 0.0: 563, 0.5: 90, 3.0: 27, 2.0: 79}",0.562212
demo_persona,'=551/1090,0.505505,"{1.0: 331, 0.0: 551, 0.5: 97, 3.0: 25, 2.0: 81}",0.568203
demo,'=570/1090,0.522936,"{1.0: 317, 0.0: 570, 0.5: 82, 3.0: 33, 2.0: 83}",0.574194
persona,'=533/1090,0.488991,"{1.0: 329, 0.0: 533, 0.5: 105, 3.0: 29, 2.0: 89}",0.595853


# Get distribution of responses by question

In [60]:
viz_filename = f"date0826_v10post_midterm_{survey_name}_history_run1.json"

In [61]:
import pandas as pd
from src.utils import get_file_from_s3
from tabulate import tabulate

test_q_keys = test_q_keys_mapping[survey_name]
survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
sub_df = survey_df[survey_df['key'].isin(test_q_keys)][['key', 'question', 'references']]

print(tabulate(sub_df, headers="keys"))

    key                 question                                                                                                                                                                      references
--  ------------------  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------  ----------------------------------------------------------------------------------------------------------------------------------------
 1  WORLDDANGER_W26     Compared to 50 years ago, do you think                                                                                                                                        ['We live in a safer world', 'We live in a more dangerous world', 'The world we live in is neither safer nor more dangerous', 'Refused']
17  REASONGUNC_W26      Please indicate whether the following is a major reason, a minor reason, or not a reason why you 

In [62]:
q_key_to_q_text = dict(zip(sub_df['key'], sub_df['question']))
q_texts = [q_key_to_q_text[key] for key in test_q_keys]

print(q_key_to_q_text)
print(q_texts)

{'WORLDDANGER_W26': 'Compared to 50 years ago, do you think', 'REASONGUNC_W26': 'Please indicate whether the following is a major reason, a minor reason, or not a reason why you own a gun. For sport shooting, including target shooting and trap and skeet', 'GUNIDENTITY_W26': 'How important, if at all, is being a gun owner to your overall identity?', 'GUNRESPNOKIDSB_W26': 'Thinking about gun owners who do not have children in their home how important do you think it is for them to: Keep all of their guns unloaded', 'GUNRESPKIDSC_W26': 'Thinking about gun owners who have children in their home, how important do you think it is for them to: Take gun safety courses'}
['Thinking about gun owners who do not have children in their home how important do you think it is for them to: Keep all of their guns unloaded', 'Compared to 50 years ago, do you think', 'How important, if at all, is being a gun owner to your overall identity?', 'Please indicate whether the following is a major reason, a mino

In [63]:
import textwrap
import json
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
res = [{"pred": [], "gold": []} for _ in range(5)]
for filename in os.listdir(f'opinions_qa/output/{survey_name}'):

    with open(f"opinions_qa/output/{survey_name}/{viz_filename}", 'r') as f:
        data = json.load(f)

    for entry in data:
        q_idx = entry['q_idx']
        prediction = entry['prediction']
        gold_answer = entry['gold_answer']
        res[q_idx]['pred'].append(prediction)
        res[q_idx]['gold'].append(gold_answer)



# Create dataframes for visualization
dataframes = []
for i in range(5):
    df_pred = pd.Series(res[i]['pred']).value_counts().reset_index()
    df_pred.columns = ['Answer', 'Count']
    df_pred['Type'] = 'Prediction'
    df_pred['Question'] = f'Question {i+1}'
    
    df_gold = pd.Series(res[i]['gold']).value_counts().reset_index()
    df_gold.columns = ['Answer', 'Count']
    df_gold['Type'] = 'Gold'
    df_gold['Question'] = f'Question {i+1}'
    
    dataframes.append(pd.concat([df_pred, df_gold]))

# Plotting
sns.set(style="whitegrid")

# Function to plot the data
def plot_question(df, question_idx):
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(data=df, x="Answer", y="Count", hue="Type", palette=["#a6cee3", "#1f78b4"], alpha=0.7)
    for p in ax.patches:
        height = p.get_height()
        if height > 0:
            ax.annotate(f'{int(height)}', 
                        (p.get_x() + p.get_width() / 2., height), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 10), 
                        textcoords = 'offset points',
                        fontsize=10)
    wrapped_title = "\n".join(textwrap.wrap(q_texts[question_idx], width=80))
    plt.title(wrapped_title, fontsize=16, pad=20)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel('Answer', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Type', title_fontsize='13', fontsize='11')
    plt.show()

# Draw the graphs one by one
for i in range(5):
    plot_question(dataframes[i], i)


FileNotFoundError: [Errno 2] No such file or directory: 'opinions_qa/output/American_Trends_Panel_W26/date0826_v10post_midterm_American_Trends_Panel_W26_history_run1.json'